In [1]:
source('../src/load_data.r')
source('../src/multiplot.r')

In [2]:
final_ames_df$X <- NULL
head(final_ames_df)
t(dim(final_ames_df))

MSSubClass.20,MSSubClass.30,MSSubClass.40,MSSubClass.45,MSSubClass.50,MSSubClass.60,MSSubClass.70,MSSubClass.75,MSSubClass.80,MSSubClass.85,⋯,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,FirstFlrSF,GrLivArea,GarageYrBlt,GarageArea,SalePrice
-0.760586,-0.2233679,-0.05255887,-0.09128733,-0.3318133,1.9746741,-0.2076167,-0.1055563,-0.2021419,-0.1181804,⋯,1.0509849,0.8820760,0.7205018,0.08917398,0.10842138,-0.8026317,0.5315369,1.02265460,0.2771098,0.5639683
1.313870,-0.2233679,-0.05255887,-0.09128733,-0.3318133,-0.5060637,-0.2076167,-0.1055563,-0.2021419,-0.1181804,⋯,0.1692304,-0.4188065,0.7552734,0.20492237,0.22641726,0.4214040,-0.3785471,-0.09204807,0.2390487,0.2168273
-0.760586,-0.2233679,-0.05255887,-0.09128733,-0.3318133,1.9746741,-0.2076167,-0.1055563,-0.2021419,-0.1181804,⋯,0.9860786,0.8342087,0.6806608,0.28181796,0.13033862,-0.5752722,0.6618057,0.94060083,0.2997008,0.7378659
-0.760586,-0.2233679,-0.05255887,-0.09128733,-0.3318133,-0.5060637,4.8132480,-0.1055563,-0.2021419,-0.1181804,⋯,-1.8680714,-0.7103048,0.5941378,0.32144248,0.07065950,-0.4377884,0.5437750,0.81736628,0.3115320,-0.4330156
-0.760586,-0.2233679,-0.05255887,-0.09128733,-0.3318133,1.9746741,-0.2076167,-0.1055563,-0.2021419,-0.1181804,⋯,0.9536011,0.7384023,0.7125017,0.30382402,0.19684307,0.1146148,1.2836237,0.89954319,0.3689426,1.0183392
-0.760586,-0.2233679,-0.05255887,-0.09128733,3.0116663,-0.5060637,-0.2076167,-0.1055563,-0.2021419,-0.1181804,⋯,0.7258029,0.4984665,0.7243605,-0.06527209,0.08633152,-1.0317819,-0.1501048,0.61156359,0.2483025,-0.3799439


1451,403


In [3]:
install.packages('mlbench')
library(mlbench)
library(caret)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Loading required package: lattice


In [ ]:
set.seed(42)

In [ ]:
final_ames_df$SalePrice <- sale_price

In [ ]:
train_index <- (createDataPartition(final_ames_df$SalePrice, p =0.6, list=FALSE, times=1))


In [ ]:
train = final_ames_df[train_index,]
validate = final_ames_df[-train_index,]
write.csv(train, "train.csv")
write.csv(validate, "validate.csv")

In [ ]:
plot(density(train$SalePrice))
plot(density(validate$SalePrice))

### Variable Ranking by Recursive Feature Elimination

In [ ]:
install.packages('Hmisc')
install.packages('randomForest')

In [ ]:
library(Hmisc)
library(randomForest)

In [ ]:
normalization <- preProcess(ames)
subsets <- c(1:5, 10, 15, 20, 25)

In [ ]:
ctrl <- rfeControl(functions = lmFuncs,
                   method = "repeatedcv",
                   repeats = 10,
                   verbose = FALSE)

In [ ]:
options(warn=-1)

In [ ]:
lmProfile <- rfe(train[,1:401], train$SalePrice,
                 sizes = subsets,
                 rfeControl = ctrl)

In [ ]:
options(warn=0)
lmProfile

### Variable Ranking by Linear Modeling

In [ ]:
library(broom)

In [ ]:
lmRating <- lm(train$SalePrice ~ ., data=train)
lmCoefs <- tidy(lmRating)
head(lmCoefs)

In [ ]:
lmCoefs$abs_est <- abs(lmCoefs$estimate)
lmCoefsAbs <- lmCoefs[-c(2:5)]
lmCoefsAbs <- lmCoefsAbs[-1,]
head(lmCoefsAbs[order(-lmCoefsAbs$abs_est), ], 10)

In [ ]:
top_ten_df <- data.frame(train$BsmtQual.TA,
                         train$BsmtQual.Gd,
                         train$BsmtQual.Ex,
                         train$TotalBsmtSF,
                         train$BsmtFinSF1,
                         train$BsmtQual.Fa,
                         train$BsmtFinType1.GLQ,
                         train$GarageType.Attchd,
                         train$GarageType.Detchd,
                         train$BsmtFinType1.ALQ,
                         train$SalePrice)

In [ ]:
write.csv(top_ten_df, "top_ten_df.csv")